In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
data=pd.read_csv('C:/Users/astle/Downloads/gps20.cat',sep='\n',skiprows=4)

In [3]:
data

,Long Lat RA Dec Sprob Fpeak Fint RMS Maj Min PA Field OldGPS
0,339.739+0.419 16 44 12.282 -45 10 17.79 0.17 ...
1,339.968+0.555 16 44 28.207 -44 54 36.14 0.06 ...
2,339.980-0.539 16 49 14.776 -45 36 32.25 0.02 ...
3,340.025-0.002 16 47 04.495 -45 13 41.51 0.60 ...
4,340.051-0.231 16 48 09.875 -45 21 25.84 0.02 ...
...,...
5040,119.012+0.029 00 17 13.202 +62 38 25.25 0.02 ...
5041,119.071+0.027 00 17 44.083 +62 38 45.97 0.02 ...
5042,119.339-0.504 00 20 35.979 +62 09 08.99 0.02 ...
5043,119.450+0.401 00 20 37.076 +63 03 50.31 0.23 ...


In [4]:
data.iloc[:,0][913][:13].split('-')[0]
data.iloc[:,0][913][81:86]

' 1.03'

In [5]:
query,size=[],[]
for i in range (len(data)):
    l=float(data.iloc[:,0][i][:7])
    b=float(data.iloc[:,0][i][8:13])
    c=float(data.iloc[:,0][i][81:86])
    if l<48.5 and l>5:
        if abs(b)<0.8:
            query.append([l,data.iloc[:,0][i][7:13]])
            size.append(c)

In [6]:
size[:10]

[2.18, 1.06, 0.0, 1.49, 0.71, 0.0, 2.0, 6.18, 2.85, 41.4]

In [22]:
indices=[]
for i,x in enumerate(size):
    if x<5 or x>30:
        indices.append(i)
indices=sorted(indices,reverse=True)#reverse sort so that the indices of the elements to remove from query isnt affected

In [23]:
for i in indices:
    del query[i]
    del size[i]

In [70]:
testq=query[60:80]
tests=size[60:80]

In [71]:
for i in testq:
    i[1]=float(i[1])

In [72]:
driver=webdriver.Firefox()
driver.get('https://third.ucllnl.org/cgi-bin/gpscutout')
script='arguments[0].value=arguments[1]'

In [73]:
j=0
for i in range(len(testq)):
    try:
        RA=driver.find_element_by_name('RA')
        Size=driver.find_element_by_name('ImageSize')
        driver.execute_script(script, RA,'{} {}'.format(testq[i][0],testq[i][1]))
        driver.execute_script(script,Size,'{}'.format(tests[i]))
        
        exg=driver.find_element_by_name('Equinox')
        driver.execute_script(script,exg,'{}'.format('Galactic'))
        
        download=driver.find_elements_by_name('ImageType')[2]
        download.click()
        time.sleep(2)
        submit=driver.find_element_by_name('.submit')
        submit.click()
    except NoSuchElementException:
        print('missing file with Galactic coordinates: {}'.format(testq[i]))
        driver.get('https://third.ucllnl.org/cgi-bin/gpscutout')
        time.sleep(1)
        j+=1
        continue
print('missing files percentage: '+ str((round(100*j*len(testq)**-1,2))) +'%')

missing files percentage: 0.0%
